In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
indexing_distinct_tables = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True)
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [ ]:
tf.__version__

In [ ]:
test_articles = pd.read_csv('../dataset/data_articles_test.csv', delimiter=',')

In [ ]:
embedding_model = gs.models.FastText.load('../train_embedding_models/fasttext_embedding_50d_all_signals')

In [ ]:
indexing_distinct_tables.indices.close(index='distinct_tables')
indexing_distinct_tables.indices.put_settings(index='distinct_tables', body={"index": {"similarity": {"default": {"type": "classic"}}}})
indexing_distinct_tables.indices.open(index='distinct_tables')

In [ ]:
# "tablePgID"
# "tablePgTitle"
# "tablePgFullText"
# "tablePgMetaDescription"
# "tablePgSummary"
# "tablePgKeywords"
# "tableSectionTitle"
# "tableCaption"
# "tableHeader"
# "tableBody"

In [ ]:
def search_indexing(query):
    
    result= indexing_distinct_tables.search(
        index="distinct_tables", 
        body = {
        "_source": ["tablePgID","tablePgTitle","tablePgSummary","tablePgKeywords"],
        "from" : 0,
        "size" : 100,
        "query": {
            "multi_match":{
              "type": "most_fields",
              "query":    query, 
              "fields": ["tablePgFullText"] 
            }
        }
    })
    
    return result

In [ ]:
def get_accuracy(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    
    for table_ID in ranked_tables_ID:
        
        if table_ID[0] == ID_goal:
    
            accuracy = 1
            break;

    return accuracy

In [ ]:
MAX_PAD_TITLE = 55

def sequence_padding(X_DIM, value):
    
    value_padding = np.pad(value, ((0,MAX_PAD_TITLE - X_DIM),(0,0)), 'constant')
    
    return value_padding

In [ ]:
def create_embedding(value):
    
    if value == "":
        
        value = [""]
    
    else:
        
        value = tknzr.tokenize(str(value))
    
    if len(value) < MAX_PAD_TITLE:
        
        embedding = embedding_model.wv[value]
        embedding = embedding.astype('float16')
        
        padding_embedding = sequence_padding(embedding.shape[0],embedding)
        
        return padding_embedding
        
    else:
        
        embedding = embedding_model.wv[value[0:MAX_PAD_TITLE]]
        embedding = embedding.astype('float16')
        
        return embedding

In [ ]:
def search_index(query):
    
    tables_index = []

    result_index = search_indexing(query)
        
    for hit in result_index['hits']['hits']:
    
        table_ID = hit['_source']['tablePgID']
        
        table_page_title = hit['_source']['tablePgTitle']
        
        table_page_main_passage = hit['_source']['tablePgSummary']
        
        table_page_keywords = hit['_source']['tablePgKeywords']
    
        tables_index.append([table_ID,table_page_title,table_page_main_passage, table_page_keywords])
    
    return tables_index

In [ ]:
ranking_model = tf.keras.models.load_model('coattention_model_title_main_passage_keywords_1_1_11_0.9621.h5')

In [ ]:
test_articles.head(1)

In [ ]:
result = []

def run_search(k):
    
    TOP_K = k
    accuracy = []
    
    for i, row in tqdm(test_articles.iterrows()):
        
        article_ID = row['page_id']
        article_title_text = row['page_title']
        article_main_passage_text = row['meta_description']
        article_keywords_text = row['keywords'] 
        
        query = row['page_title']+" "+row['meta_description']+" "+row['keywords']
        
        article_title = []
        article_main_passage = []
        article_keywords = []
        table_title = []
        table_main_passage = []
        table_keywords = []
        ranked_tables_model = []

        ranked_tables_index = search_index(query)
        
        if len(ranked_tables_index) > 0:
            
            article_title_embedding = create_embedding(article_title_text)
            article_main_passage_embedding = create_embedding(article_main_passage_text)
            article_keywords_embedding = create_embedding(article_keywords_text)
            
            for table_ID, index_table_title, index_table_main_passage, index_table_keywords in (ranked_tables_index):
                       
                table_title_embedding = create_embedding(str(index_table_title))
                table_main_passage_embedding = create_embedding(str(index_table_main_passage))
                table_keywords_embedding = create_embedding(str(index_table_keywords))
                
                article_title.append(article_title_embedding)
                article_main_passage.append(article_main_passage_embedding)
                article_keywords.append(article_keywords_embedding)
                
                table_title.append(table_title_embedding)
                table_main_passage.append(table_main_passage_embedding)
                table_keywords.append(table_keywords_embedding)

            article_title = np.array(article_title, dtype='float16')
            article_main_passage = np.array(article_main_passage, dtype='float16')
            article_keywords = np.array(article_keywords, dtype='float16')
            
            table_title = np.array(table_title, dtype='float16')
            table_main_passage = np.array(table_main_passage, dtype='float16')
            table_keywords = np.array(table_keywords, dtype='float16')
           
            table_ranking_model = ranking_model.predict([article_title, article_main_passage, article_keywords, table_title, table_main_passage, table_keywords])

            for i in range(0,len(table_ranking_model)):

                ranked_tables_model.append([ranked_tables_index[i][0],ranked_tables_index[i][1],table_ranking_model[i][0]]) 

            data_frame = pd.DataFrame(ranked_tables_model, columns = ['table_ID', 'table_title','table_ranking']) 
            data_frame_sorting = data_frame.sort_values('table_ranking', ascending=False)   
#             final_ranked_tables = data_frame_sorting.iloc[0:TOP_K,0:1].values
            
            selected_top = data_frame_sorting.head(TOP_K)
            min_score = selected_top['table_ranking'].min()
            draw_tables_socres = data_frame_sorting[data_frame_sorting['table_ranking'] >= min_score]
            final_ranked_tables = draw_tables_socres.iloc[:,0:1].values
        
            accuracy.append(get_accuracy(article_ID, final_ranked_tables))

    result.append(["Acc@"+str(k),str(round(np.mean(accuracy),4))])

In [ ]:
accuracy_K = [1,5,10,20]

for k in accuracy_K:
     
    run_search(k)

In [ ]:
result